## IEEE-CV options
- [IEEE - CV options](https://www.kaggle.com/kyakovlev/ieee-cv-options)
- 모델링은 lightGBM
- 여러 cross validation 모델 모으고, 시계열 데이터 반영(뒷부분)
- 이것도 결국 [data minification](https://www.kaggle.com/kyakovlev/ieee-data-minification) 참고한 커널. 일단 원 데이터로 해 보기

In [5]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, warnings, random, datetime

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import lightgbm as lgb

import math
warnings.filterwarnings('ignore')

for dirname, _, filenames in os.walk("../input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input\df_id.pkl
../input\df_pca.pkl
../input\df_test.pkl
../input\df_train.pkl
../input\df_trans.pkl
../input\sample_submission.csv
../input\test.pkl
../input\test_0823.pkl
../input\test_identity.csv
../input\test_transaction.csv
../input\train_0823.pkl
../input\train_identity.csv
../input\train_transaction.csv


In [2]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [3]:
SEED = 41
seed_everything(SEED)
TARGET = 'isFraud'
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

In [4]:
# These parameters we will keep untouched
# for each lgbm model
# the unique param that we will look at
# is n_estimators
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':20000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': SEED,
                    'early_stopping_rounds':100, 
                } 

In [15]:
print('Load data')
train_df = pd.read_pickle("../input/df_train.pkl")
test_df = pd.read_pickle("../input/df_test.pkl")

Load data


In [8]:
# We will prepare simulation here
# Last month will be our test test 
# train_df['DT_M'] = train_df['TransactionDT'].apply(
#     lambda x: (START_DATE + datetime.timedelta(seconds = x)))
# train_df['DT_M'] = (train_df['DT_M'].dt.year-2017)*12 + train_df['DT_M'].dt.month 

# test_df = train_df[
#     train_df['DT_M'] == train_df['DT_M'].max()].reset_index(drop=True)
# train_df = train_df[
#     train_df['DT_M'] < train_df['DT_M'].max()].reset_index(drop=True)
    
print('Shape control:', train_df.shape, test_df.shape)

Shape control: (501214, 435) (89326, 435)


In [9]:
test_df['DT_M'].head() # 원래 train에서 DT_M이 17(max)인 것만 모음

0    17
1    17
2    17
3    17
4    17
Name: DT_M, dtype: int64

In [10]:
train_df['DT_M'].head()

0    12
1    12
2    12
3    12
4    12
Name: DT_M, dtype: int64

In [16]:
# object data 타입 변경
for col in list(train_df):
    if train_df[col].dtype=='O':
        print(col)
        train_df[col] = train_df[col].fillna('unseen_before_label')
        test_df[col]  = test_df[col].fillna('unseen_before_label')
        
        train_df[col] = train_df[col].astype(str)
        test_df[col] = test_df[col].astype(str)
        
        le = LabelEncoder()
        le.fit(list(train_df[col])+list(test_df[col]))
        train_df[col] = le.transform(train_df[col])
        test_df[col]  = le.transform(test_df[col])
        
        train_df[col] = train_df[col].astype('category')
        test_df[col] = test_df[col].astype('category')

ProductCD
card4
card6
P_emaildomain
R_emaildomain
M1
M2
M3
M4
M5
M6
M7
M8
M9
id_12
id_15
id_16
id_23
id_27
id_28
id_29
id_30
id_31
id_33
id_34
id_35
id_36
id_37
id_38
DeviceType
DeviceInfo


In [13]:
# Remove Some Features
rm_cols = [
    'TransactionID','TransactionDT', # These columns are pure noise right now
    TARGET,                          # Not target in features))
    'DT_M'                           # Column that we used to simulate test set
]

# Remove V columns (for faster training)
rm_cols += ['V'+str(i) for i in range(1,340)] #V1 ~ V339 제외시킴
# print(rm_cols)

# Final features
features_columns = [col for col in list(train_df) if col not in rm_cols]

## CV(cross-validation)

In [14]:
RESULTS = test_df[['TransactionID',TARGET]]

# We will always use same number of splits
# for training model
# Number of splits depends on data structure
# and in our case it is better to use 
# something in range 5-10
# 5 - is a common number of splits
# 10+ is too much (we will not have enough diversity in data)
# Here we will use 3 for faster training
# but you can change it by yourself
N_SPLITS = 3

### 1. No validation